In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from __future__ import print_function, division

In [4]:
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.optimizers import Adam

Using TensorFlow backend.


In [5]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import ama as a

In [8]:
BASE_PATH = '../data/'

In [9]:
import cPickle as pickle
train_df = df = pickle.load(open(BASE_PATH+ 'df_train.pkl', 'rb'))

In [20]:
import xgboost as xgb
### TRAIN XGBOOST ###

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
train_df.head()

,image_name,tags,path,haze,primary,agriculture,clear,water,habitation,road,...,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down,fdata
0,train_0,haze primary,../data/train-full-jpg/train_0.jpg,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[10696302.0, 9545515.0, 9921551.0, 0.0, 163.21..."
1,train_1,agriculture clear primary water,../data/train-full-jpg/train_1.jpg,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,"[12409906.0, 11698348.0, 12497611.0, 0.0, 189...."
2,train_2,clear primary,../data/train-full-jpg/train_2.jpg,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,"[13882853.0, 12908011.0, 12985172.0, 0.0, 211...."
3,train_3,clear primary,../data/train-full-jpg/train_3.jpg,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,"[13342388.0, 12435209.0, 13174508.0, 0.0, 203...."
4,train_4,agriculture clear habitation primary road,../data/train-full-jpg/train_4.jpg,0,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,"[12953690.0, 13745953.0, 15432086.0, 0.0, 197...."


In [17]:
train_df.columns[3:-1]

Index([u'haze', u'primary', u'agriculture', u'clear', u'water', u'habitation',
       u'road', u'cultivation', u'slash_burn', u'cloudy', u'partly_cloudy',
       u'conventional_mine', u'bare_ground', u'artisinal_mine', u'blooming',
       u'selective_logging', u'blow_down'],
      dtype='object')

In [43]:
xtrain = np.vstack(train_df['fdata'].map(lambda x: np.array(x)).values)

In [49]:
cvs1 = {}
for c in train_df.columns[3:-1]:
    print(c)
    y_mean = train_df[c].mean(); print(y_mean)
    params = {
    'objective' : 'binary:logistic', # this outputs probability to be a 1
    'eval_metric':'auc', # area under curve
    'eta':0.3, #learning rate, default 0.3
    'max_depth':4,
    'base_score':y_mean,
    'subsample':1,
    'colsample_bytree':0.8,
    'colsample_bylevel':0.8,
    #'gamma':0,
    #'min_child_weight':6,
    'alpha':0.5
    }
    dtrain = xgb.DMatrix(xtrain, train_df[c].values)
    cv = xgb.cv(params, dtrain, 
                   nfold=5, 
                   num_boost_round=500, 
                   early_stopping_rounds=50, 
                   verbose_eval=10, 
                   show_stdv=False)
    cvs1[c] = cv

haze
0.0666271399985
[0]	train-auc:0.951264	test-auc:0.943799
[10]	train-auc:0.973819	test-auc:0.963087
[20]	train-auc:0.978528	test-auc:0.964998
[30]	train-auc:0.982054	test-auc:0.965477
[40]	train-auc:0.984861	test-auc:0.965651
[50]	train-auc:0.98714	test-auc:0.965649
[60]	train-auc:0.98913	test-auc:0.965806
[70]	train-auc:0.990919	test-auc:0.965674
[80]	train-auc:0.992297	test-auc:0.965536
[90]	train-auc:0.993589	test-auc:0.965415
[100]	train-auc:0.994786	test-auc:0.965406
[110]	train-auc:0.995743	test-auc:0.965259
primary
0.926727438919
[0]	train-auc:0.94315	test-auc:0.936596
[10]	train-auc:0.979811	test-auc:0.971507
[20]	train-auc:0.985027	test-auc:0.973535
[30]	train-auc:0.988169	test-auc:0.974659
[40]	train-auc:0.99033	test-auc:0.975115
[50]	train-auc:0.992254	test-auc:0.975406
[60]	train-auc:0.993772	test-auc:0.975738
[70]	train-auc:0.995089	test-auc:0.976043
[80]	train-auc:0.996091	test-auc:0.976309
[90]	train-auc:0.996936	test-auc:0.976427
[100]	train-auc:0.997582	test-auc:0.

[80]	train-auc:1	test-auc:0.858754
[90]	train-auc:1	test-auc:0.858039
[100]	train-auc:1	test-auc:0.856543
[110]	train-auc:1	test-auc:0.854272
cloudy
0.0516070061019
[0]	train-auc:0.966032	test-auc:0.960453
[10]	train-auc:0.993444	test-auc:0.987963
[20]	train-auc:0.995214	test-auc:0.988935
[30]	train-auc:0.996465	test-auc:0.989335
[40]	train-auc:0.997419	test-auc:0.989518
[50]	train-auc:0.998072	test-auc:0.989772
[60]	train-auc:0.998655	test-auc:0.989917
[70]	train-auc:0.999067	test-auc:0.990119
[80]	train-auc:0.999369	test-auc:0.990214
[90]	train-auc:0.999587	test-auc:0.990307
[100]	train-auc:0.999759	test-auc:0.990276
[110]	train-auc:0.999864	test-auc:0.99029
[120]	train-auc:0.999927	test-auc:0.990339
[130]	train-auc:0.999966	test-auc:0.990299
[140]	train-auc:0.999984	test-auc:0.990373
[150]	train-auc:0.999992	test-auc:0.990392
[160]	train-auc:0.999997	test-auc:0.990414
[170]	train-auc:0.999999	test-auc:0.990431
[180]	train-auc:1	test-auc:0.990507
[190]	train-auc:1	test-auc:0.990557
[

In [56]:
for c in train_df.columns[3:-1]:
    print(len(cvs1[c]))

63
130
293
147
500
89
113
100
71
229
109
90
48
132
35
66
46


In [60]:
for c in train_df.columns[3:-1]:
    print(c)
    y_mean = train_df[c].mean(); print(y_mean)
    params = {
    'objective' : 'binary:logistic', # this outputs probability to be a 1
    'eval_metric':'auc', # area under curve
    'eta':0.3, #learning rate, default 0.3
    'max_depth':4,
    'base_score':y_mean,
    'subsample':1,
    'colsample_bytree':0.8,
    'colsample_bylevel':0.8,
    #'gamma':0,
    #'min_child_weight':6,
    'alpha':0.5
    }
    dtrain = xgb.DMatrix(xtrain, train_df[c].values)
    model = xgb.train(dict(params), dtrain, num_boost_round=len(cvs1[c]), 
                  evals=[(dtrain, 'eval')])
    pickle.dump(model, open(BASE_PATH + 'xgb-models/' + c + '-model.pkl', 'wb'), protocol=2)

haze
0.0666271399985
[0]	eval-auc:0.9517
[1]	eval-auc:0.959745
[2]	eval-auc:0.963625
[3]	eval-auc:0.966206
[4]	eval-auc:0.967367
[5]	eval-auc:0.968627
[6]	eval-auc:0.969007
[7]	eval-auc:0.970068
[8]	eval-auc:0.970897
[9]	eval-auc:0.971699
[10]	eval-auc:0.972194
[11]	eval-auc:0.972595
[12]	eval-auc:0.9733
[13]	eval-auc:0.973787
[14]	eval-auc:0.974245
[15]	eval-auc:0.974807
[16]	eval-auc:0.975367
[17]	eval-auc:0.976086
[18]	eval-auc:0.976346
[19]	eval-auc:0.976901
[20]	eval-auc:0.97718
[21]	eval-auc:0.977518
[22]	eval-auc:0.977646
[23]	eval-auc:0.977854
[24]	eval-auc:0.978209
[25]	eval-auc:0.97883
[26]	eval-auc:0.979158
[27]	eval-auc:0.979485
[28]	eval-auc:0.979855
[29]	eval-auc:0.980242
[30]	eval-auc:0.980646
[31]	eval-auc:0.980818
[32]	eval-auc:0.98107
[33]	eval-auc:0.981254
[34]	eval-auc:0.981585
[35]	eval-auc:0.981777
[36]	eval-auc:0.982011
[37]	eval-auc:0.98221
[38]	eval-auc:0.982298
[39]	eval-auc:0.982508
[40]	eval-auc:0.982769
[41]	eval-auc:0.983025
[42]	eval-auc:0.983301
[43]	eva

[160]	eval-auc:0.974792
[161]	eval-auc:0.974939
[162]	eval-auc:0.975141
[163]	eval-auc:0.975206
[164]	eval-auc:0.975378
[165]	eval-auc:0.975439
[166]	eval-auc:0.975587
[167]	eval-auc:0.975883
[168]	eval-auc:0.976055
[169]	eval-auc:0.976209
[170]	eval-auc:0.976443
[171]	eval-auc:0.976636
[172]	eval-auc:0.976767
[173]	eval-auc:0.976981
[174]	eval-auc:0.977118
[175]	eval-auc:0.977271
[176]	eval-auc:0.977456
[177]	eval-auc:0.977624
[178]	eval-auc:0.977668
[179]	eval-auc:0.977803
[180]	eval-auc:0.977853
[181]	eval-auc:0.977928
[182]	eval-auc:0.978057
[183]	eval-auc:0.978154
[184]	eval-auc:0.978244
[185]	eval-auc:0.978331
[186]	eval-auc:0.978475
[187]	eval-auc:0.978538
[188]	eval-auc:0.978637
[189]	eval-auc:0.978747
[190]	eval-auc:0.97881
[191]	eval-auc:0.978927
[192]	eval-auc:0.979071
[193]	eval-auc:0.979253
[194]	eval-auc:0.979377
[195]	eval-auc:0.97955
[196]	eval-auc:0.979739
[197]	eval-auc:0.97988
[198]	eval-auc:0.979977
[199]	eval-auc:0.980147
[200]	eval-auc:0.980307
[201]	eval-auc:0.98

[70]	eval-auc:0.903465
[71]	eval-auc:0.904447
[72]	eval-auc:0.905852
[73]	eval-auc:0.906189
[74]	eval-auc:0.907032
[75]	eval-auc:0.90785
[76]	eval-auc:0.90861
[77]	eval-auc:0.909256
[78]	eval-auc:0.909976
[79]	eval-auc:0.910711
[80]	eval-auc:0.911596
[81]	eval-auc:0.91207
[82]	eval-auc:0.912501
[83]	eval-auc:0.913127
[84]	eval-auc:0.913628
[85]	eval-auc:0.914016
[86]	eval-auc:0.914641
[87]	eval-auc:0.91492
[88]	eval-auc:0.915436
[89]	eval-auc:0.916087
[90]	eval-auc:0.916486
[91]	eval-auc:0.916778
[92]	eval-auc:0.917136
[93]	eval-auc:0.917663
[94]	eval-auc:0.917879
[95]	eval-auc:0.918296
[96]	eval-auc:0.919051
[97]	eval-auc:0.919431
[98]	eval-auc:0.920149
[99]	eval-auc:0.921054
[100]	eval-auc:0.921345
[101]	eval-auc:0.921639
[102]	eval-auc:0.922348
[103]	eval-auc:0.922614
[104]	eval-auc:0.923167
[105]	eval-auc:0.924025
[106]	eval-auc:0.924899
[107]	eval-auc:0.92566
[108]	eval-auc:0.926529
[109]	eval-auc:0.926854
[110]	eval-auc:0.927439
[111]	eval-auc:0.927926
[112]	eval-auc:0.928103
[11

[415]	eval-auc:0.991554
[416]	eval-auc:0.991609
[417]	eval-auc:0.991766
[418]	eval-auc:0.991786
[419]	eval-auc:0.991926
[420]	eval-auc:0.991951
[421]	eval-auc:0.992001
[422]	eval-auc:0.992119
[423]	eval-auc:0.992189
[424]	eval-auc:0.992283
[425]	eval-auc:0.992391
[426]	eval-auc:0.992518
[427]	eval-auc:0.99264
[428]	eval-auc:0.992674
[429]	eval-auc:0.992715
[430]	eval-auc:0.992768
[431]	eval-auc:0.992809
[432]	eval-auc:0.992882
[433]	eval-auc:0.992935
[434]	eval-auc:0.992979
[435]	eval-auc:0.993022
[436]	eval-auc:0.993048
[437]	eval-auc:0.993022
[438]	eval-auc:0.993131
[439]	eval-auc:0.993173
[440]	eval-auc:0.993234
[441]	eval-auc:0.993257
[442]	eval-auc:0.993297
[443]	eval-auc:0.99331
[444]	eval-auc:0.993326
[445]	eval-auc:0.993417
[446]	eval-auc:0.993472
[447]	eval-auc:0.993584
[448]	eval-auc:0.993642
[449]	eval-auc:0.993719
[450]	eval-auc:0.993773
[451]	eval-auc:0.993786
[452]	eval-auc:0.993831
[453]	eval-auc:0.993872
[454]	eval-auc:0.993918
[455]	eval-auc:0.994005
[456]	eval-auc:0.9

[65]	eval-auc:0.92597
[66]	eval-auc:0.926792
[67]	eval-auc:0.927663
[68]	eval-auc:0.928675
[69]	eval-auc:0.928856
[70]	eval-auc:0.929048
[71]	eval-auc:0.929385
[72]	eval-auc:0.929709
[73]	eval-auc:0.930185
[74]	eval-auc:0.930584
[75]	eval-auc:0.93114
[76]	eval-auc:0.93153
[77]	eval-auc:0.932512
[78]	eval-auc:0.93346
[79]	eval-auc:0.933588
[80]	eval-auc:0.934361
[81]	eval-auc:0.934991
[82]	eval-auc:0.936023
[83]	eval-auc:0.936878
[84]	eval-auc:0.937425
[85]	eval-auc:0.938315
[86]	eval-auc:0.93929
[87]	eval-auc:0.939673
[88]	eval-auc:0.940051
[89]	eval-auc:0.940827
[90]	eval-auc:0.941316
[91]	eval-auc:0.941665
[92]	eval-auc:0.942137
[93]	eval-auc:0.94293
[94]	eval-auc:0.943243
[95]	eval-auc:0.943839
[96]	eval-auc:0.944259
[97]	eval-auc:0.944563
[98]	eval-auc:0.945077
[99]	eval-auc:0.945418
slash_burn
0.00516317102695
[0]	eval-auc:0.874589
[1]	eval-auc:0.899207
[2]	eval-auc:0.915124
[3]	eval-auc:0.926374
[4]	eval-auc:0.93387
[5]	eval-auc:0.936712
[6]	eval-auc:0.938389
[7]	eval-auc:0.94387

[21]	eval-auc:0.983912
[22]	eval-auc:0.984219
[23]	eval-auc:0.984408
[24]	eval-auc:0.984808
[25]	eval-auc:0.985092
[26]	eval-auc:0.985234
[27]	eval-auc:0.985598
[28]	eval-auc:0.985891
[29]	eval-auc:0.986217
[30]	eval-auc:0.98653
[31]	eval-auc:0.986937
[32]	eval-auc:0.987348
[33]	eval-auc:0.987664
[34]	eval-auc:0.987775
[35]	eval-auc:0.98796
[36]	eval-auc:0.988242
[37]	eval-auc:0.98842
[38]	eval-auc:0.988627
[39]	eval-auc:0.988795
[40]	eval-auc:0.988909
[41]	eval-auc:0.989104
[42]	eval-auc:0.989281
[43]	eval-auc:0.989542
[44]	eval-auc:0.989645
[45]	eval-auc:0.989714
[46]	eval-auc:0.989826
[47]	eval-auc:0.98999
[48]	eval-auc:0.990151
[49]	eval-auc:0.990287
[50]	eval-auc:0.990503
[51]	eval-auc:0.990594
[52]	eval-auc:0.990763
[53]	eval-auc:0.990976
[54]	eval-auc:0.991104
[55]	eval-auc:0.991235
[56]	eval-auc:0.991451
[57]	eval-auc:0.991544
[58]	eval-auc:0.991666
[59]	eval-auc:0.991711
[60]	eval-auc:0.99184
[61]	eval-auc:0.991908
[62]	eval-auc:0.991967
[63]	eval-auc:0.992107
[64]	eval-auc:0.

[32]	eval-auc:0.989137
[33]	eval-auc:0.989424
[34]	eval-auc:0.989764
selective_logging
0.00839941698164
[0]	eval-auc:0.884702
[1]	eval-auc:0.912919
[2]	eval-auc:0.923245
[3]	eval-auc:0.930097
[4]	eval-auc:0.933205
[5]	eval-auc:0.93714
[6]	eval-auc:0.943958
[7]	eval-auc:0.9484
[8]	eval-auc:0.951744
[9]	eval-auc:0.955301
[10]	eval-auc:0.957486
[11]	eval-auc:0.959103
[12]	eval-auc:0.961049
[13]	eval-auc:0.962716
[14]	eval-auc:0.964355
[15]	eval-auc:0.965827
[16]	eval-auc:0.968824
[17]	eval-auc:0.97076
[18]	eval-auc:0.973817
[19]	eval-auc:0.976951
[20]	eval-auc:0.978017
[21]	eval-auc:0.980095
[22]	eval-auc:0.982061
[23]	eval-auc:0.983264
[24]	eval-auc:0.983792
[25]	eval-auc:0.984914
[26]	eval-auc:0.986356
[27]	eval-auc:0.987
[28]	eval-auc:0.988107
[29]	eval-auc:0.988931
[30]	eval-auc:0.989734
[31]	eval-auc:0.990037
[32]	eval-auc:0.991286
[33]	eval-auc:0.991703
[34]	eval-auc:0.991888
[35]	eval-auc:0.992237
[36]	eval-auc:0.992909
[37]	eval-auc:0.993246
[38]	eval-auc:0.994023
[39]	eval-auc:0.

In [61]:
print("done")

done


In [62]:
df.head()

,image_name,tags,path,haze,primary,agriculture,clear,water,habitation,road,...,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down,fdata
0,train_0,haze primary,../data/train-full-jpg/train_0.jpg,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[10696302.0, 9545515.0, 9921551.0, 0.0, 163.21..."
1,train_1,agriculture clear primary water,../data/train-full-jpg/train_1.jpg,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,"[12409906.0, 11698348.0, 12497611.0, 0.0, 189...."
2,train_2,clear primary,../data/train-full-jpg/train_2.jpg,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,"[13882853.0, 12908011.0, 12985172.0, 0.0, 211...."
3,train_3,clear primary,../data/train-full-jpg/train_3.jpg,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,"[13342388.0, 12435209.0, 13174508.0, 0.0, 203...."
4,train_4,agriculture clear habitation primary road,../data/train-full-jpg/train_4.jpg,0,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,"[12953690.0, 13745953.0, 15432086.0, 0.0, 197...."


In [63]:
d_feat_train = xgb.DMatrix(xtrain)

In [68]:
for c in train_df.columns[3:-1]:
    print(c)
    model = pickle.load(open(BASE_PATH + 'xgb-models/' + c + '-model.pkl', 'rb'))
    train_df[c+'_pred'] = model.predict(d_feat_train)

haze
primary
agriculture
clear
water
habitation
road
cultivation
slash_burn
cloudy
partly_cloudy
conventional_mine
bare_ground
artisinal_mine
blooming
selective_logging
blow_down


In [75]:
train_df.drop(['tags','path','fdata'], 1).to_csv(BASE_PATH + 'train_pred.csv', index=False)